In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = '/content/drive/My Drive/CS529'
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 3.5MB/s 
     |████████████████████████████████| 645kB 46.5MB/s 


In [0]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from sklearn.metrics.pairwise import cosine_similarity
import re
import pandas as pd
import numpy as np
from tqdm import tqdm

In [0]:
def parse_file(filepath):
	lang = []
	data = []

	with open(filepath, 'r') as file_object:
		line = file_object.readline()
		f = open(path + "/my_test.txt","w")
		while line:
		    a =  re.findall(r"[\w']+", line)
		    # a = line.strip().split(' ')
		    for i in range(len(a)):
		       if i % 2 != 1:
		         data.append(a[i])
		       if i % 2 != 0:
		         lang.append(a[i])
		    res = [a[j] for j in range(len(a)) if j % 2 != 1] 
		    # print(res)
		    listToStr = ' '.join([str(elem) for elem in res]) 
		    f.write(listToStr + "\n") 

		    line = file_object.readline()

		f.close()
	return data, lang

In [0]:
def fun():
  data, lang = parse_file(path + "/test.txt")
  with open(path + '/my_test.txt') as f:
    lines = f.readlines()
  idx = 0
  col = []
  for i in range(768):
    col.append(str(i))
  df = pd.DataFrame(columns = col)
  skipped = []
  ## Load pre-trained model tokenizer (vocabulary)
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  for i in tqdm(range(len(lines))):
    text = lines[i]
    #print(text)
    
    marked_text = "[CLS] " + text + " [SEP]"


    tokenized_text = tokenizer.tokenize(marked_text)
    #tokenized_text = marked_text.split()
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # Load pre-trained model (weights)
    model = BertModel.from_pretrained('bert-base-uncased')

    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()

    try:

      # Predict hidden states features for each layer
      with torch.no_grad():
          encoded_layers, _ = model(tokens_tensor, segments_tensors)

      #print ("Number of layers:", len(encoded_layers))
      layer_i = 0

      #print ("Number of batches:", len(encoded_layers[layer_i]))
      batch_i = 0

      #print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
      token_i = 0

      #print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))

      # Holds the list of 12 layer embeddings for each token
      # Will have the shape: [# tokens, # layers, # features]
      token_embeddings = [] 

      # For each token in the sentence...
      for token_i in range(len(tokenized_text)):
        
        # Holds 12 layers of hidden states for each token 
        hidden_layers = [] 
        
        # For each of the 12 layers...
        for layer_i in range(len(encoded_layers)):
          
          # Lookup the vector for `token_i` in `layer_i`
          vec = encoded_layers[layer_i][batch_i][token_i]
          
          hidden_layers.append(vec)
          
        token_embeddings.append(hidden_layers)

      # Sanity check the dimensions:
      #print ("Number of tokens in sequence:", len(token_embeddings))
      #print ("Number of layers per token:", len(token_embeddings[0]))

      #concatenated_last_4_layers = [torch.cat((layer[-1], layer[-2], layer[-3], layer[-4]), 0) for layer in token_embeddings] # [number_of_tokens, 3072]

      summed_last_1_layers = [torch.sum(torch.stack(layer)[-1:], 0) for layer in token_embeddings] # [number_of_tokens, 768]
      
      for token in range(1,len(tokenized_text)-1):
        if tokenized_text[token][0] == '#':
          continue
        df.loc[len(df)] = summed_last_1_layers[token].numpy()
      idx = idx+1
      # Compare "bank" as in "bank robber" to "bank" as in "river bank"
      #different_bank = cosine_similarity(summed_last_4_layers[10].reshape(1,-1), summed_last_4_layers[19].reshape(1,-1))[0][0]
    except:
      skipped.append(idx)
      idx=idx+1

    # Compare "bank" as in "bank robber" to "bank" as in "bank vault" 
    #same_bank = cosine_similarity(summed_last_4_layers[10].reshape(1,-1), summed_last_4_layers[6].reshape(1,-1))[0][0]
  df.to_csv(path + "/test_data.csv", index=False)
  return skipped

In [0]:
skipped = fun()

100%|██████████| 2390/2390 [9:24:22<00:00, 15.00s/it]


In [0]:
with open(path + '/my.txt') as f:
    lines = f.readlines()
data, lang = parse_file(path + "/train.txt")
s=0
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
langtoken = []
sum=0
for i in tqdm(range(len(lines))):
    text = lines[i]
    marked_text = "[CLS] " + text + " [SEP]"
    words = marked_text.split()
    tokenized_text = tokenizer.tokenize(marked_text)
    #print(words)
    #print(tokenized_text)
    tokenidx = 1
    for token in range(1,len(tokenized_text)-1):
        if tokenized_text[token][0] == '#':
          langtoken.append(0)
          continue
        sum=sum+1
        langtoken.append(1)


100%|██████████| 1000/1000 [00:00<00:00, 1192.12it/s]

In [0]:
print(len(langtoken))

57074


In [0]:
print(skipped)

[]


In [0]:
dfs = pd.DataFrame(skipped)

In [0]:
dfs.to_csv(path+'/skipped.csv', index = False)

In [0]:
with open(path + '/my.txt') as f:
    lines = f.readlines()
data, lang = parse_file(path + "/train.txt")
skipped = pd.read_csv(path+'/skipped.csv')
labels = []
idx=0
for i in range(len(lines)):
  text = lines[i]
  words = text.split()
  if i in skipped['0'].tolist():
    idx=idx+len(words)
    continue
  for token in words:
    labels.append(lang[idx])
    idx = idx+1

In [0]:
dfl = pd.DataFrame(labels)
dfl.to_csv(path+'/labels_2.csv', index = False)

In [0]:
data, lang = parse_file(path + "/test.txt")
dft = pd.DataFrame(lang)
dft.to_csv(path + '/test_labels.csv',index = False)